In [1]:
import cv2
import numpy as np
from skimage import measure, color, io
from os.path import join
import io
import pandas as pd
from skimage import img_as_ubyte, exposure
from skimage.measure import regionprops_table
import matplotlib.pyplot as plt
from skimage.segmentation import clear_border
import csv
import os
import skimage
from PIL import Image
import colorsys

In [60]:
imagePath = 'preprocessedImages/rainbow/inducedRainbowKI67/thresholded/19-23/aorta05/230703_C5RB 19-23_aorta5_Ki67_stitch unmix denoise_red_thresholded8bit.tif'
originalImagePath = "preprocessedImages/rainbow/inducedRainbowKI67/stitched/19-23/aorta05/230703_C5RB 19-23_aorta5_Ki67_stitch unmix denoise_red.tif"
ki67ImagePath = "preprocessedImages/rainbow/inducedRainbowKI67/stitched/19-23/aorta05/230703_C5RB 19-23_aorta5_Ki67_stitch unmix denoise_ki67.tif"
distTransformMultiplyer = 0.01
cellposeFileName = "cellposeSegm_RedP19-23_aorta05_50Thres.npy"
savePath = 'segmentations/inducedRainbowKI67/old/19-23/aorta05'

In [61]:
img1 = cv2.imread(imagePath)
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

In [62]:
ret1, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
kernel = np.ones((3,3),np.uint8)

In [63]:
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)
#we use the thresh bc opening gets rid of too many segmentations
opening = thresh
opening = clear_border(opening)

In [64]:
sure_bg = cv2.dilate(opening,kernel,iterations=10)
opening8Bit = opening.astype(np.uint8)
dist_transform = cv2.distanceTransform(opening8Bit,cv2.DIST_L2,3)
ret2, sure_fg = cv2.threshold(dist_transform,distTransformMultiplyer*dist_transform.max(),255,0)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
ret3, markers = cv2.connectedComponents(sure_fg)
markers = markers+10
markers[unknown==255] = 0
markers = cv2.watershed(img1,markers)

In [65]:
img1[markers == -1] = [0,255,255]
img2 = color.label2rgb(markers, bg_label=0)

In [66]:
#connect touching clusters
markersNew = np.copy(markers)
markersNew[markersNew == 10] = 0
minus1 = np.where(markersNew == -1)
for i in range(len(minus1[0])):
    minus1y = minus1[0][i]
    minus1x = minus1[1][i]
    if (minus1y != 0) & (minus1y != (markersNew.shape[0] -1)) & (minus1x != 0) & (minus1x != (markersNew.shape[1]-1)):
        numbers = np.unique([markersNew[minus1y - 1,minus1x], markersNew[minus1y + 1,minus1x], markersNew[minus1y,minus1x-1], markersNew[minus1y,minus1x +1]])
        uniqueMasks  = set(numbers) - set([-1, 0])
        masks = list(uniqueMasks)
        if len(uniqueMasks) > 1:
            masks = list(uniqueMasks)
            markersNew = np.where(markersNew == masks[1], masks[0], markersNew)
        if 0 not in numbers:
            markersNew[minus1y, minus1x] = masks[0]

In [67]:
cellposeFile = {}
#read in image
originalImg = skimage.io.imread(originalImagePath)
nucleiImg = skimage.io.imread(ki67ImagePath)
zeros = np.zeros(originalImg.shape, dtype="uint8")
#8bit normalize & merge
dapi8bitNormalized= img_as_ubyte(exposure.rescale_intensity(nucleiImg))
rainbow8bitNormalized = img_as_ubyte(exposure.rescale_intensity(originalImg))
dapi8bitNormalized = np.asarray(dapi8bitNormalized)
dapi8bitNormalized = dapi8bitNormalized*0.1
dapi8bitNormalized = dapi8bitNormalized.astype(rainbow8bitNormalized.dtype)
merged8bitNormalized = cv2.merge([rainbow8bitNormalized, dapi8bitNormalized, zeros])
#define masks and outlines
masks = np.where(markersNew==-1, 0, markersNew)
outlines = np.where(markersNew==-1,1,0)
#save in dic
cellposeFile['masks'] = masks
cellposeFile['img'] = merged8bitNormalized
cellposeFile['outlines'] = outlines
np.save(join(savePath, cellposeFileName),cellposeFile)